# Linear Regression with MCMC Simulated Annealing

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from mcmc_optimization import optimize_via_mcmc
from models import LinearLayer
from data_samplers import BatchSampler
from proposal_generators import LinearLayerGaussianProposalGenerator

## Pure interpolation

In [ ]:
# Generate data.

true_function = lambda x: 3*x+5
len_data = 1000
x_vals = np.linspace(-10,10,len_data)
y_vals = true_function(x_vals)

plt.title("Deterministic Linear Data")
plt.scatter(x_vals, y_vals, s=3)
plt.show()

In the example below we use a Gaussian random walk for our proposal Markov Chain.

In [ ]:
initial_params = {'w': 0.2, 'b': 6}
data_sampler = BatchSampler(X=x_vals, y=y_vals)
proposal_generator = LinearLayerGaussianProposalGenerator(scale=1.1)

best_params, history = optimize_via_mcmc(LinearLayer, initial_params, data_sampler, proposal_generator, mean_squared_error)

In [ ]:
print(f"Proportion of proposals accepted: {history['acceptance_ratio']:.4f}")
print(f"Best parameters found: w={best_params['w']}, b={best_params['b']}")
print(f"Best achieved loss: {min(history['loss_values']):.4f}")

In [ ]:
w_vals = history['parameter_values']['w']
b_vals = history['parameter_values']['b']

plt.plot(w_vals, b_vals, '-->', label="Path")
plt.plot(best_params['w'], best_params['b'], 'o', label="Estimated Minimizer")
plt.plot([3], [5], 'o', label="True Minimizer")
plt.xlabel("w")
plt.ylabel("b")
plt.legend()
plt.title("MCMC Weight Selection")
plt.show()

## Noisy regression (univariate contrived)

In [ ]:
# Generate data.

true_function = lambda x: 3*x+5
len_data = 1000
x_vals = np.linspace(-10,10,len_data)
y_vals = true_function(x_vals) + np.random.normal(scale=2, size=len(x_vals))

plt.title("Noisy Linear Data")
plt.scatter(x_vals, y_vals, s=5)
plt.show()

In [ ]:
# Fit linear regression model the standard way.

from sklearn.linear_model import LinearRegression

ground_truth_linear = LinearRegression().fit(x_vals.reshape(-1, 1), y_vals)

ground_truth_linear.coef_, ground_truth_linear.intercept_

In [ ]:
# Fit linear regression model using MCMC.

initial_params = {'w': 0.2, 'b': 6}
data_sampler = BatchSampler(X=x_vals, y=y_vals)
proposal_generator = LinearLayerGaussianProposalGenerator(scale=1.1)

best_params, history = optimize_via_mcmc(LinearLayer, initial_params, data_sampler,
                                         proposal_generator, mean_squared_error, num_iterations=10000)

In [ ]:
print(f"Proportion of proposals accepted: {history['acceptance_ratio']:.4f}")
print(f"Best parameters found: w={best_params['w']}, b={best_params['b']}")
print(f"Best achieved loss: {min(history['loss_values']):.4f}")

In [ ]:
w_vals = history['parameter_values']['w']
b_vals = history['parameter_values']['b']

plt.plot(w_vals, b_vals, '-->', label="Path")
plt.plot(best_params['w'], best_params['b'], 'o', label="Estimated Minimizer")
plt.plot([3], [5], 'o', label="True Minimizer")
plt.xlabel("w")
plt.ylabel("b")
plt.legend()
plt.title("MCMC Weight Selection")
plt.show()

In [ ]:
# Plot OLS regression against MCMC method.

plt.scatter(x_vals, y_vals, s=5, alpha=0.3)
plt.plot(x_vals, best_params['w'] * x_vals + best_params['b'], '--', label="MCMC Learned", c='r')
plt.plot(x_vals, ground_truth_linear.predict(x_vals.reshape(-1, 1)), '--', label="OLS Regression", c='k')

plt.legend()
plt.show()

## Real-life linear regression (multivariate)



In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

X, y = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
ground_truth_linear = LinearRegression().fit(X_train, y_train)
actual_w, actual_b = ground_truth_linear.coef_, ground_truth_linear.intercept_

print(f"w = {np.round(actual_w, 3)}\n b = {actual_b}")
print(f"MSE = {mean_squared_error(y_train, ground_truth_linear.predict(X_train))}")

In [ ]:
initial_params = {'w': np.random.normal(size=X_train.shape[1]), 'b': np.random.uniform(-30, 30)}
data_sampler = BatchSampler(X=X_train, y=y_train)
proposal_generator = LinearLayerGaussianProposalGenerator(scale=1.1, decay=(1 - 1e-7))

best_params, history = optimize_via_mcmc(LinearLayer, initial_params, data_sampler,
                                         proposal_generator, mean_squared_error, 
                                         beta=1e-4, num_iterations=25000, batch_size=150)

In [ ]:
print(f"Proportion of proposals accepted: {history['acceptance_ratio']:.4f}")
print(f"Best parameters found: w={np.round(best_params['w'], 3)}, b={np.round(best_params['b'], 3)}")
print(f"Best achieved loss: {min(history['loss_values']):.4f}")

In [ ]:
plt.title("Computed Loss Values")
plt.plot(np.log(history['loss_values']))
plt.ylabel("$log(MSE)$")
plt.xlabel("Iteration")
plt.show()

In [ ]:
w_vals = np.row_stack(history['parameter_values']['w'])
b_vals = np.row_stack(history['parameter_values']['b'])

w_vals.shape, b_vals.shape

In [ ]:
plt.title("Normed Difference from Known Minimizer")
plt.plot((np.linalg.norm(w_vals - actual_w, axis=1) + np.abs(b_vals - actual_b).ravel()))
plt.ylabel("$||\mathbf{\\theta^{*}} - \mathbf{\hat{\\theta}}||$")
plt.xlabel("Iteration")
plt.show()

In linear regression, at least, we are not approaching the optimizer with this method.